<a href="https://colab.research.google.com/github/Roshano1/SQL_week3/blob/main/MasterSchool/SQL_week3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Original Author : UTSAV AWASTHI

Modified By: JOE GANSER

<h2> Importing Datasets </h2>
Let's import the relevant datasets on which we will run SQL queries. Press Ctrl+Enter to run the codes in each of the boxes below.<br><br>
I highly advice you to download the excel file (available <a href="https://github.com/ItsAwSM/SQL-for-Data-Analytics/blob/main/sales_database.xlsx?raw=true">here</a>) which has the schema and the data which we will query in this assignment. Spend some time to go through the schema, understand the relationships between different datasets and then analyze all the datasets in excel once. 

In [1]:
!pip install xlrd
import pandas as pd
import sqlite3
conn = sqlite3.connect(':memory:')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [112]:
#Importing customer dataset
customer = pd.read_excel("https://github.com/ItsAwSM/SQL-for-Data-Analytics/blob/main/sales_database.xlsx?raw=true",sheet_name="customer_dim")
customer.to_sql('customer',conn,if_exists='replace',index=False)
customer.head()

,cust_id,first_name,last_name,email,phone,primary_pincode,gender,dob,joining_date
0,10000001,Rahul,Gupta,rahul.gupta@gmail.com,8960722319,110001,male,02-12-1977,19-02-2016
1,10000002,Abhinav,Sharma,abhinav.sharma@gmail.com,8857852863,400001,male,20-09-1995,01-10-2016
2,10000003,Anubhav,Patel,anubhav.patel@gmail.com,8492146048,560001,male,26-12-1986,16-08-2018
3,10000004,Neha,Verma,neha.verma@gmail.com,8631861914,600001,female,20-01-2002,28-07-2009
4,10000005,John,Bernard,john.bernard@gmail.com,8215391844,500001,male,NaN,05-03-2017


In [113]:
#Importing product dataset
product = pd.read_excel("https://github.com/ItsAwSM/SQL-for-Data-Analytics/blob/main/sales_database.xlsx?raw=true",sheet_name="product_dim")
product.to_sql('product',conn,if_exists='replace',index=False)
product.head()

,product_id,product_name,brand,category,procurement_cost_per_unit,mrp
0,12345,HP XYZ Mouse,HP,mouse,650,1500
1,12346,HP 241H,HP,laptop,45000,80000
2,12347,HP 8GB Pendrive,HP,pendrive,350,800
3,12348,Dell ABC Mouse,Dell,mouse,350,1100
4,12349,Dell AX420,Dell,laptop,30000,75000


# Using the two examples above as imports, fill in the commented out values to load the orders table from github.

In [114]:
#Importing order dataset
table_link = "https://github.com/ItsAwSM/SQL-for-Data-Analytics/blob/main/sales_database.xlsx?raw=true"
sheet_name = "order_dim"
table_name = "my_order"

order = pd.read_excel("https://github.com/ItsAwSM/SQL-for-Data-Analytics/blob/main/sales_database.xlsx?raw=true",sheet_name="order_dim")
order.to_sql('my_order',conn,if_exists='replace',index=False)
order.head()

,order_id,order_type,cust_id,order_date,delivery_date,tot_units,displayed_selling_price_per_unit,total_amount_paid,product_id,delivery_person_id,payment_type,delivery_pincode
0,10000000001,buy,10000009,01-01-2020,03-01-2020,2,73500,138180,12349,1000003,cash,500001
1,10000000002,buy,10000004,01-01-2020,04-01-2020,9,67500,577125,12349,1000005,cash,700001
2,10000000003,buy,10000012,01-01-2020,02-01-2020,2,60750,119070,12349,1000002,netbanking,400001
3,10000000004,buy,10000012,01-01-2020,03-01-2020,8,736,5824,12347,1000010,cash,600001
4,10000000005,buy,10000008,01-01-2020,06-01-2020,1,60750,57105,12349,1000003,debit,500001


<b>Q0.</b> How many customers are male and how many are female?

In [115]:
query0 = '''
SELECT gender,count(*)
FROM customer
GROUP BY gender

'''
pd.read_sql_query(query0,conn)

,gender,count(*)
0,female,4
1,male,10


<b>Q1.</b> How many customers do not have DOB information available?

In [116]:
query1 = '''
SELECT COUNT(cust_id) 
FROM Customer
WHERE dob is null;

'''

pd.read_sql_query(query1,conn)

,COUNT(cust_id)
0,2


<b>Q2.</b> How many customers are there in each pincode and gender combination?

In [117]:
query2 = """

SELECT primary_pincode,	gender,	COUNT(*) AS cust_cnt
FROM Customer
GROUP BY primary_pincode, Gender;

"""

pd.read_sql_query(query2,conn)

,primary_pincode,gender,cust_cnt
0,110001,male,3
1,400001,male,2
2,500001,female,1
3,500001,male,1
4,560001,female,1
5,560001,male,1
6,600001,female,1
7,600001,male,1
8,700001,female,1
9,700001,male,2


# Q3 For each `payment type`, find the avg, total and max values for `tot_units` and `total_amount_paid` columns of the `orders` table where the `order_type = 'buy'`. 

In [118]:
query = """
SELECT payment_type,
       AVG(tot_units) AS avg_tot_units,
       SUM(tot_units) AS total_tot_units,
       MAX(tot_units) AS max_tot_units,
       AVG(total_amount_paid) AS avg_total_amount_paid,
       SUM(total_amount_paid) AS total_total_amount_paid,
       MAX(total_amount_paid) AS max_total_amount_paid
FROM 'order'
WHERE order_type = 'buy'
GROUP BY payment_type;

"""

pd.read_sql_query(query,conn)

,payment_type,avg_tot_units,total_tot_units,max_tot_units,avg_total_amount_paid,total_total_amount_paid,max_total_amount_paid
0,cash,5.465021,1328,10,129749.255144,31529069,721280
1,credit,5.524904,1442,10,121767.965517,31781439,768240
2,debit,5.157025,1248,10,105574.632231,25549061,698400
3,netbanking,5.311024,1349,10,118246.283465,30034556,683850


# Q4: From the `product` table, using a `CASE` statement to identify products from HP vs products by Dell, count how many HP products there are and how many Dell products.

In [119]:
query = """
SELECT 
    CASE 
        WHEN brand = 'HP' THEN 'HP'
        WHEN brand = 'Dell' THEN 'Dell'
    END AS company,
    COUNT(*) AS count
FROM product
WHERE brand IN ('HP', 'Dell')
GROUP BY company;

"""

pd.read_sql_query(query,conn)

,company,count
0,Dell,3
1,HP,3


# Q5: Join the customer, orders and product tables together. Examine each table (above) and determine that columns need to be joined. Use an inner join to connect the 3 tables.

In [120]:
query5 = """
SELECT *
FROM customer
INNER JOIN my_order ON customer.cust_id = my_order.cust_id
INNER JOIN product ON my_order.product_id = product.product_id;

"""


pd.read_sql_query(query5,conn)

,cust_id,first_name,last_name,email,phone,primary_pincode,gender,dob,joining_date,order_id,...,product_id,delivery_person_id,payment_type,delivery_pincode,product_id,product_name,brand,category,procurement_cost_per_unit,mrp
0,10000001,Rahul,Gupta,rahul.gupta@gmail.com,8960722319,110001,male,02-12-1977,19-02-2016,10000000009,...,12348,1000006,cash,400001,12348,Dell ABC Mouse,Dell,mouse,350,1100
1,10000001,Rahul,Gupta,rahul.gupta@gmail.com,8960722319,110001,male,02-12-1977,19-02-2016,10000000031,...,12346,1000001,debit,560001,12346,HP 241H,HP,laptop,45000,80000
2,10000001,Rahul,Gupta,rahul.gupta@gmail.com,8960722319,110001,male,02-12-1977,19-02-2016,10000000047,...,12349,1000009,credit,400001,12349,Dell AX420,Dell,laptop,30000,75000
3,10000001,Rahul,Gupta,rahul.gupta@gmail.com,8960722319,110001,male,02-12-1977,19-02-2016,10000000076,...,12346,1000005,netbanking,700001,12346,HP 241H,HP,laptop,45000,80000
4,10000001,Rahul,Gupta,rahul.gupta@gmail.com,8960722319,110001,male,02-12-1977,19-02-2016,10000000091,...,12350,1000006,netbanking,400001,12350,Dell 8GB Pendrive,Dell,pendrive,250,850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1045,10000014,Amanpreet,Singh,amanpreet.singh@yahoo.com,8153506539,110001,male,10-03-1980,29-08-2008,10000000919,...,12347,1000008,credit,110001,12347,HP 8GB Pendrive,HP,pendrive,350,800
1046,10000014,Amanpreet,Singh,amanpreet.singh@yahoo.com,8153506539,110001,male,10-03-1980,29-08-2008,10000000932,...,12346,1000007,credit,700001,12346,HP 241H,HP,laptop,45000,80000
1047,10000014,Amanpreet,Singh,amanpreet.singh@yahoo.com,8153506539,110001,male,10-03-1980,29-08-2008,10000000932,...,12346,1000007,credit,700001,12346,HP 241H,HP,laptop,45000,80000
1048,10000014,Amanpreet,Singh,amanpreet.singh@yahoo.com,8153506539,110001,male,10-03-1980,29-08-2008,10000000939,...,12345,1000008,netbanking,110001,12345,HP XYZ Mouse,HP,mouse,650,1500
